# Import

In [1]:
import csv
import os
import re
import sys

import numpy as np
import pandas as pd
from nltk import download, word_tokenize
from nltk.corpus import stopwords
from stop_words import get_stop_words
from tqdm import tqdm

from urlextract import URLExtract

## Add configuration file

In [2]:
sys.path.append("/home/jovyan/core/config/")
sys.path.append("/home/jovyan/core/util/")

In [3]:
from ALL import config 
from util import *

## Set condition

In [4]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [5]:
s3 = S3Manager()

# Read data

In [6]:
df_path = s3.download("DataShaping/TweetTopic/master.csv")

In [7]:
df = pd.read_csv(df_path[0], index_col=0)

In [8]:
labels_path = s3.download("DataShaping/TweetTopic/class.csv")

In [9]:
with open(labels_path[0], mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

# Word tokinize

In [10]:
download('punkt')
download("stopwords")

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## delete URL

In [11]:
def format_tweet(tweet):
    # mask web urls
    urls = extractor.find_urls(tweet)
    for url in urls:
        tweet = tweet.replace(url, "")
    # format twitter account
    tweet = re.sub(r"\b(\s*)(@[\S]+)\b", r'\1{\2@}', tweet)
    return tweet

In [12]:
extractor = URLExtract()
df["text"] = df.text.progress_apply(format_tweet)

100%|██████████| 6997/6997 [00:11<00:00, 591.07it/s]


In [13]:
df["words"] = df.text.progress_apply(word_tokenize)

100%|██████████| 6997/6997 [00:02<00:00, 3078.09it/s]


In [14]:
stop_words_add = ["would", "could", "should"]
stop_char = ["==", "--", "\'s", "''", "n't", "``","..", "...", "....", "'m", "'ve","'re", "'d", "'ll", "", "-+", "+-", "_/", "||", "__", "/|", "//", "{", "}"]
stop_words = set(stopwords.words("english") + get_stop_words("english") + stop_words_add + stop_char)

In [15]:
#     一文字以下の単語とstop_word, stop_charを削除
df["words_nonstop"] = df.words.progress_apply(
    lambda words: [
        word for word in words if word.lower() not in stop_words if len(word)> 1
    ]
)

100%|██████████| 6997/6997 [00:00<00:00, 92067.84it/s]


In [16]:
df.words = df.words.progress_apply(lambda words: " ".join(words))
df.words_nonstop = df.words_nonstop.progress_apply(
    lambda words: " ".join(words)
)

100%|██████████| 6997/6997 [00:00<00:00, 417425.90it/s]


# output

## make file

In [17]:
with open(make_filepath(f"../../temporary/Preprocessing/TweetTopic/class.csv"), "w") as f:
    writer = csv.writer(f)
    writer.writerow(class_labels)
df.to_csv(make_filepath(f"../../temporary/Preprocessing/TweetTopic/master.csv"))

## upload file

In [18]:
s3.upload("../../temporary/Preprocessing/TweetTopic", "Preprocessing/TweetTopic/")

'../../temporary/Preprocessing/TweetTopic'

In [19]:
s3.delete_local_all()

../../temporary/Preprocessing/TweetTopic
/home/jovyan/temporary/DataShaping/TweetTopic/master.csv
/home/jovyan/temporary/DataShaping/TweetTopic/class.csv


In [20]:
shutil.rmtree("../../nltk_data/")